In [1]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
sns.set(rc={'axes.facecolor':'white', 'figure.facecolor':'white'})

folder = '../data'
merge_file = 'pilot1'
plot_folder = '../plots/pilot1'

experiments = [
    # {
    #     'pagetimes': 'PageTimes-2021-12-09.csv',
    #     'main_part': 'main_part_pilot_2021-12-09.csv',
    #     'folder': '../data/prepilot'
    # },
    # {
    #     'pagetimes': 'PageTimes-2021-12-13.csv',
    #     'main_part': 'main_part_pilot_2021-12-13.csv',
    #     'folder': '../data/prepilot2'
    # }, 
    # {
    #     'pagetimes': 'PageTimes-2021-12-20.csv',
    #     'main_part': 'main_part_pilot_cont_2021-12-20.csv',
    #     'folder': '../data/prepilot4'
    # }, 
    {
        'pagetimes': 'PageTimes-2022-01-13.csv',
        'main_part': 'main_part_pilot_cont_2022-01-13.csv',
        'folder': '../data/prepilot5'
    }, 
    {
        'pagetimes': 'PageTimes-2022-01-27.csv',
        'main_part': 'main_part_pilot_cont_2022-01-27.csv',
        'folder': '../data/pilot1'
    }, 
]


session_map = {
    'sdvfxj22': 'flextimes', 
    'n8op39bm': 'fixedtimes',
    'qeklctjv': 'continous_exp',
    'a4ly7fp4': 'continous_cheap',
    'yi3xaim0': 'trail_rounds',
    'sdvfxj22': 'trail_rounds_2'
}



exp_pool = {
    'flextimes': 'discrete', 
    'fixedtimes': 'discrete',
    'continous_exp': 'continous',
    'continous_cheap': 'continous',
    'trail_rounds': 'trail_rounds'
}



dfs = []
dfts = []
for exp in experiments:
    df = pd.read_csv(os.path.join(exp['folder'], exp['main_part']))
    dft = pd.read_csv(os.path.join(exp['folder'], exp['pagetimes']))
    df['experiment_name'] = df['session.code'].map(session_map)
    dft['experiment_name'] = dft['session_code'].map(session_map)
    df['experiment_pool'] = df['experiment_name'].map(exp_pool)
    dfs.append(df)
    dfts.append(dft)
df = pd.concat(dfs)
dft = pd.concat(dfts)
df['experiment_name'].value_counts()

trail_rounds    960
Name: experiment_name, dtype: int64

In [2]:
df['session.code'].value_counts()

mqqsqhav    3200
dbcrara4    2720
yi3xaim0     960
1g4f5wxy     160
xl4vfraf     160
hmjq6293     160
Name: session.code, dtype: int64

In [3]:
df.head()

,participant.id_in_session,participant.code,participant.label,participant._is_bot,participant._index_in_pages,participant._max_page_index,participant._current_app_name,participant._current_page_name,participant.time_started_utc,participant.visited,...,group.eligible_ids,subsession.round_number,session.code,session.label,session.mturk_HITId,session.mturk_HITGroupId,session.comment,session.is_demo,experiment_name,experiment_pool
0,1,z2kav0uk,NaN,0,0,186,NaN,NaN,NaN,0,...,"[1,2,3,4,5]",1,1g4f5wxy,NaN,NaN,NaN,NaN,1,NaN,NaN
1,2,i9836y9t,NaN,0,0,186,NaN,NaN,NaN,0,...,"[1,2,3,4,5]",1,1g4f5wxy,NaN,NaN,NaN,NaN,1,NaN,NaN
2,3,16lxok93,NaN,0,0,186,NaN,NaN,NaN,0,...,"[1,2,3,4,5]",1,1g4f5wxy,NaN,NaN,NaN,NaN,1,NaN,NaN
3,4,v6d1zdj9,NaN,0,0,186,NaN,NaN,NaN,0,...,"[1,2,3,4,5]",1,1g4f5wxy,NaN,NaN,NaN,NaN,1,NaN,NaN
4,5,dimn40h6,NaN,0,0,186,NaN,NaN,NaN,0,...,"[1,2,3,4,5]",1,1g4f5wxy,NaN,NaN,NaN,NaN,1,NaN,NaN


In [4]:
where = (
    (df['player.role'] != 'manager') &
    (~df['player.role'].isnull()) &
    (df['participant._current_page_name'] == 'FinalScreen')
)

df = df[where]

assert df.groupby(['session.code', 'subsession.round_number', 'participant.code'])['participant.label'].count().max() == 1

In [5]:

max_player = 4

def get_punishment(row):
    member_idx = row['player.role'][-1]
    return row[f"group.punishment{member_idx}"]

rename = {
    'session.code': 'session', 
    'group.id_in_subsession': 'group_id', 
    'subsession.round_number': 'round_number',
    'participant.code': 'participant_code',
    'player.contribution': 'contribution',
    'player.no_input': 'player_no_input',
    'player.contribution': 'contribution',
    'player.punishment': 'punishment',
    'group.no_input_manager': 'manager_no_input',
    'group.total_contribution': 'total_contribution',
    'group.common_good': 'common_good',
    'group.total_punishment': 'total_punishment',
    'episode': 'episode',
    'experiment_name': 'experiment_name',
    'player.role': 'player_role',
    'player.payoff': 'payoff',
    'experiment_pool': 'experiment_pool'
}

df['episode'] = (df['subsession.round_number'] - 1) // 8 + 1


df['player.punishment'] = df.apply(get_punishment, axis=1)

df = df[rename.keys()]
df = df.rename(columns=rename)

df['round_number'] = df['round_number'] % 8 
# dft = dft[dft['page_name'] == 'Results']
# merge_on = ['experiment_name', 'round_number', 'participant_code']

df['global_group_id'] = df['session'] + ' #' + df['group_id'].astype('str')

# df = df.merge(dft[merge_on + ['epoch_time_completed']], on=merge_on, how='left')


# df['group_started'] = df.groupby(['group_id', 'session'])['epoch_time_completed'].transform('min')
# df['global_group_id'] = df['group_started'].rank(method='dense').astype(int)

# df['time_passed'] = df['epoch_time_completed'] - df.groupby('participant_code')['epoch_time_completed'].transform('min')
# df['group_duration'] = df.groupby('global_group_id')['time_passed'].transform('max')
# df['group_duration_min'] = df['group_duration'] / 60



In [6]:
groupby = ['global_group_id', 'round_number', 'episode']

check = (df['punishment'].isnull() == df['contribution'].isnull())
assert check.all(), 'If punishment is null, contribution is null as well.'
n_p_null = df['punishment'].isnull().sum()
print(f"""
There are {n_p_null} rows with null punishments. These are likely from apported games. We will remove these rows.
""")
df = df[~df['punishment'].isnull()]

check = (df['manager_no_input'] == (df['punishment'] == 0))
assert check.any(), 'Inconsistency!'

check = (df['player_no_input'] == (df['contribution'] == 0))
assert check.any(), 'Inconsistency!'

check = df.groupby(groupby)['total_contribution'].transform('first') == df.groupby(groupby)['contribution'].transform('sum')
assert check.all(), 'Sum of total contributions does not match group contribution.'


There are 44 rows with null punishments. These are likely from apported games. We will remove these rows.



In [7]:
# we might have different number of rows per round, due to some participants dropping out
check = df.groupby(groupby)['total_contribution'].count()
check.value_counts()

4    773
Name: total_contribution, dtype: int64

In [8]:
# we have different number of round per episode, due to some episodes failing early
check = df.groupby(groupby[:-1])['total_contribution'].count()
check.value_counts()

8    385
4      3
Name: total_contribution, dtype: int64

In [9]:
df.to_csv(os.path.join(folder, f'{merge_file}_player_round.csv'), index=False)

In [10]:
selected_columns = ['session', 'global_group_id','episode', 'round_number', 'participant_code', 'contribution', 'punishment', 'payoff', 'common_good']

df[selected_columns].to_csv(os.path.join(folder, f'{merge_file}_player_round_slim.csv'), index=False)